In [1]:
import os
import random
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer
from datasets import Dataset
from evaluate import load as load_metric
from itertools import islice

# 📂 Параметри
file_path = "C:\\Users\\Andriy.Bespalyy\\Desktop\\en-uk_dataset.tsv"
sample_size = 1000
chunk_size = 20_000
max_chunks = 20

# 📥 Сабсет з великим TSV-файлом
samples = []
reader = pd.read_csv(file_path, sep="\t", names=["en", "uk"], quoting=3, chunksize=chunk_size, encoding="utf-8")

for i, chunk in enumerate(islice(reader, max_chunks)):
    print(f"🔹 Обробка чанка #{i+1}")
    chunk.dropna(inplace=True)
    chunk = chunk[chunk["en"].str.len().between(4, 256) & chunk["uk"].str.len().between(4, 256)]
    
    if not chunk.empty:
        n = min(len(chunk), sample_size // 5)
        samples.append(chunk.sample(n=n, random_state=random.randint(0, 9999)))
    
    if sum(len(s) for s in samples) >= sample_size:
        break

# 🧾 Формування сабсету
if samples:
    df_sample = pd.concat(samples).sample(n=sample_size, random_state=42).reset_index(drop=True)
    print(f"✅ Сабсет сформовано: {len(df_sample)} рядків")
else:
    raise RuntimeError("❌ Не знайдено валідних рядків.")

# 📦 Завантаження uk→en моделі
model_path = "C:\\Users\\Andriy.Bespalyy\\Desktop\\marianmt-uk-en-hplt-final\\marianmt-uk-en-hplt-final"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

# 📊 BLEU-метрика
bleu = load_metric("sacrebleu")

# 📋 Dataset для HuggingFace
dataset = Dataset.from_pandas(df_sample)

# 🔁 Генерація перекладу (uk → en)
predictions = []
references = []

for sample in dataset:
    inputs = tokenizer(sample["uk"], return_tensors="pt", truncation=True, padding=True, max_length=128).to(model.device)
    with torch.no_grad():
        translated = model.generate(**inputs, max_length=128, num_beams=4)
    pred_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    predictions.append(pred_text)
    references.append([sample["en"]])  # ✅ Порівнюємо з оригінальним англ. текстом

# 📊 BLEU
result = bleu.compute(predictions=predictions, references=references)
print(f"🔹 BLEU score (uk→en): {result['score']:.2f}")


🔹 Обробка чанка #1
🔹 Обробка чанка #2
🔹 Обробка чанка #3
🔹 Обробка чанка #4
🔹 Обробка чанка #5
✅ Сабсет сформовано: 1000 рядків


C:\Python\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
C:\Python\Lib\site-packages\transformers\generation\configuration_utils.py:444: UserWarning: `num_beams` is set to None - defaulting to 1.
  warnings.warn("`num_beams` is set to None - defaulting to 1.", UserWarning)


🔹 BLEU score (uk→en): 32.14
